In [107]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [108]:
!ls /content/drive


MyDrive


In [2]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time


In [5]:
#loading dataset
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
data_sample.data

{'train': MemoryMappedTable
 Code: int64
 Name: string
 Symptoms: string
 Treatments: string
 ----
 Code: [[1,2,3,4,5,...,396,397,398,399,400]]
 Name: [["Panic disorder","Vocal cord polyp","Turner syndrome","Cryptorchidism","Ethylene glycol poisoning-1",...,"Urinary Stones (Kidney Stones)","Osteoporosis","Rheumatoid Arthritis","Type 1 Diabetes","Type 2 Diabetes"]]
 Symptoms: [["Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness","Hoarseness, Vocal Changes, Vocal Fatigue","Short stature, Gonadal dysgenesis, Webbed neck, Lymphedema","Absence or undescended testicle(s), empty scrotum, smaller or underdeveloped testicle(s), inguinal hernia, abnormal positioning of the testicle(s) (higher in the groin area)","Nausea, vomiting, abdominal pain, General malaise, weakness, Increased thirst, frequent urination",...,"Severe abdominal or back pain, blood in urine, frequent urination, pain during urination","Fragile bones, loss of height over time, back pain, 

In [10]:
data_sample.shape

{'train': (400, 4)}

In [11]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [20]:
data_df1 = pd.DataFrame(data_sample['train'])
data_df1

,Code,Name,Symptoms,Treatments
0,1,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o...","Antidepressant medications, Cognitive Behavior..."
1,2,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue","Voice Rest, Speech Therapy, Surgical Removal"
2,3,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck...","Growth hormone therapy, Estrogen replacement t..."
3,4,Cryptorchidism,"Absence or undescended testicle(s), empty scro...",Observation and monitoring (in cases of mild o...
4,5,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala...","Supportive Measures, Gastric Decontamination, ..."
...,...,...,...,...
395,396,Urinary Stones (Kidney Stones),"Severe abdominal or back pain, blood in urine,...","Pain management, increased fluid intake, medic..."
396,397,Osteoporosis,"Fragile bones, loss of height over time, back ...","Calcium and vitamin D supplements, regular exe..."
397,398,Rheumatoid Arthritis,"Joint pain, stiffness, swelling, fatigue, loss...",Medications (nonsteroidal anti-inflammatory dr...
398,399,Type 1 Diabetes,"Frequent urination, Increased thirst, Weight loss","Insulin therapy, Blood sugar monitoring, Healt..."


In [12]:
updated_data = [{'Name':item['Name'],'Symptoms' : item['Symptoms']} for item in data_sample['train']]

In [16]:
data_df = pd.DataFrame(updated_data)

In [21]:
data_df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [22]:
data_df['Symptoms'] = data_df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [23]:
data_df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [24]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [29]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  try:
    device = torch.device('cpu')
  except:
    device = torch.device('cpu')


In [30]:
device

device(type='cpu')

In [34]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

In [35]:
tokenizer

GPT2Tokenizer(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [33]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [61]:
# Dataset preparation
class LanguageDataset(Dataset):
  def __init__(self,df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient = 'records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer.encode_plus(text, return_tensors= 'pt', max_length = 128, padding = 'max_length', truncation=True)
    return tokens

  def fittest_max_length(self,df):
    max_length = max(len(max(df[self.labels[0]],key = len)),len(max(df[self.labels[1]],key = len)))
    x = 2
    while x < max_length: x = x*2
    return x


In [62]:
data_sample = LanguageDataset(df = data_df,tokenizer=tokenizer)

In [63]:
train_size = int(0.8 * len(data_sample))
val_size =  len(data_sample)-train_size
train_data, val_data = random_split(data_sample,[train_size,val_size])

In [64]:
BATCH_SIZE = 8

In [65]:
train_loader = DataLoader(train_data,batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_data,batch_size = BATCH_SIZE)

In [66]:
num_epochs = 4

In [67]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0


In [68]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(),lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [69]:
results = pd.DataFrame(columns=['epoch','transformer','batch_size','gpu','training_loss','validation_loss','epoch_duration_sec'])

In [73]:
#training loop
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  epoch_training_loss = 0
  train_iterator = tqdm(train_loader, desc = f"Training epoch {epoch+1}/{num_epochs} Batch_size :{BATCH_SIZE}, transformer :{model_name}")
  for batch in train_iterator:
    optimizer.zero_grad()
    inputs = batch['input_ids'].squeeze(1).to(device)
    targets = inputs.clone()
    outputs = model(input_ids = inputs, labels = targets)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training Loss':loss.item()})
    epoch_training_loss += loss.item()
  avg_epoch_training_loss = epoch_training_loss / len(train_iterator)
  #validation
  model.eval()
  epoch_validation_loss = 0
  total_loss = 0
  valid_iterator = tqdm(val_loader, desc = f"validation epoch {epoch+1}/{num_epochs}")
  with torch.no_grad():
    for batch in valid_iterator:
      inputs = batch['input_ids'].squeeze(1).to(device)
      targets = inputs.clone()
      outputs = model(input_ids = inputs, labels = targets)
      loss = outputs.loss
      total_loss += loss
      valid_iterator.set_postfix({'Validation Loss':loss.item()})
      epoch_validation_loss += loss.item()
  avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)
  end_time = time.time()
  epoch_duration_sec = end_time-start_time

  new_row = {
      "transformer":model_name,
      "batch_size":BATCH_SIZE,
      "gpu":gpu,
      "epoch":epoch+1,
      "training_loss":avg_epoch_training_loss,
      "validation+loss":avg_epoch_validation_loss,
      "epoch_duration_sec":epoch_duration_sec
  }

  results.loc[len(results)] = new_row
  print(f"Epoch: {epoch+1}, Validation Loss:{total_loss/len(val_loader)}")



Training epoch 1/4 Batch_size :8, transformer :distilgpt2: 100%|██████████| 40/40 [07:57<00:00, 11.94s/it, Training Loss=0.442]
validation epoch 1/4: 100%|██████████| 10/10 [00:38<00:00,  3.85s/it, Validation Loss=0.888]


Epoch: 1, Validation Loss:0.698975682258606


Training epoch 2/4 Batch_size :8, transformer :distilgpt2: 100%|██████████| 40/40 [07:59<00:00, 11.98s/it, Training Loss=0.454]
validation epoch 2/4: 100%|██████████| 10/10 [00:37<00:00,  3.72s/it, Validation Loss=0.947]


Epoch: 2, Validation Loss:0.7565123438835144


Training epoch 3/4 Batch_size :8, transformer :distilgpt2: 100%|██████████| 40/40 [08:03<00:00, 12.09s/it, Training Loss=0.423]
validation epoch 3/4: 100%|██████████| 10/10 [00:38<00:00,  3.87s/it, Validation Loss=1.01]


Epoch: 3, Validation Loss:0.8027992248535156


Training epoch 4/4 Batch_size :8, transformer :distilgpt2: 100%|██████████| 40/40 [08:10<00:00, 12.27s/it, Training Loss=0.21]
validation epoch 4/4: 100%|██████████| 10/10 [00:40<00:00,  4.01s/it, Validation Loss=1.01]

Epoch: 4, Validation Loss:0.8355714678764343


## Disease Heart burn

In [80]:
 input_str = "Heart burn"

In [81]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)
input_ids

tensor([[28541,  4245]])

In [82]:
output = model.generate(
    input_ids,
    max_length = 40,
    num_return_sequences = 1,
    do_sample = True,
    top_k = 8,
    top_p = 0.95,
    temperature = 0.5,
    repetition_penalty=1.2
)
output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[28541,  4245, 18098,    11,  7721,  2356,    11, 10802,    11, 29527,
          1272,    11, 24715, 50256]])

In [83]:
decoded_output = tokenizer.decode(output[0],skip_special_tokens=True)
decoded_output

'Heart burn sensation, chest pain, confusion, dizziness, seizures'

## Disease Diarrhea

In [88]:
 input_str = "Diarrhea"

In [97]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)
output = model.generate(
    input_ids,
    max_length = 40,
    num_return_sequences = 1,
    do_sample = True,
    top_k = 8,
    top_p = 0.95,
    temperature = 0.4,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
decoded_output = tokenizer.decode(output[0],skip_special_tokens=True)
decoded_output

'Diarrhea | Chronic constipation, abdominal pain, difficulty swallowing and speaking'

## Disease Arthritis

In [98]:
 input_str = "Arthritis"

In [102]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)
output = model.generate(
    input_ids,
    max_length = 100,
    num_return_sequences = 1,
    do_sample = True,
    top_k = 8,
    top_p = 0.95,
    temperature = 0.4,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
decoded_output = tokenizer.decode(output[0],skip_special_tokens=True)
decoded_output

'Arthritis | Joint pain, stiffness and swelling, swelling, stiffness, limited range of motion'

In [110]:
torch.save(model,'SymptomGenLM.pt')

In [111]:
torch.save(model,'drive/MyDrive/SymptomGenLM.pt')